# Data filtering & partitioning (Jonkershoek EC)

- EC tower location:
  - Lon:  18.95542123678541
  - Lat: -33.99028062649005

In [ ]:
import pandas as pd
import numpy as np
import glob

import sys
sys.path.append("C:/Users/Jonathan/Documents/Github/ecophys_utils/")
from ecophys_utils import *

In [ ]:
# Data location
project_path = './'
data_path = project_path + './'

# Input path
data_fn = '05_cec_test.csv'

In [ ]:
import numpy as np
import pandas as pd

class CECPartitioner:
    """
    Conditional Eddy-Covariance (CEC) partitioning module with wind-coordinate rotation.

    Assumes user provides high-frequency u, v, w, q, c. Coordinates can be rotated per-block
    via double rotation (Lee et al., 2004) or planar-fit method (Wilczak et al., 2001).

    Methods
    -------
    preprocess(df, time_col, u_col, v_col, w_col, q_col, c_col,
               avg_period=30, rot_method='double'):
        Compute per-block rotated coordinates, half-hourly means, and deviations (primes).
    partition(block_df, w_col='w_prime', q_col='q_prime', c_col='c_prime'):
        Apply CEC partitioning to deviations for one block.
    """

    @staticmethod
    def _double_rotate(ur, vr, wr):
        """
        Perform double rotation on arrays ur, vr, wr.
        1) Rotate about vertical axis (z) to zero mean v.
        2) Rotate about lateral axis (y, or new v) to zero mean w.
        Returns rotated (u2, v2, w2).
        """
        # Step 1: rotation angle phi to zero mean(v)
        phi = np.arctan2(np.mean(vr), np.mean(ur))
        u1 =  ur * np.cos(phi) + vr * np.sin(phi)
        v1 = -ur * np.sin(phi) + vr * np.cos(phi)
        w1 =  wr.copy()
        # Step 2: rotation angle theta to zero mean(w1)
        theta = np.arctan2(np.mean(w1), np.mean(u1))
        u2 =  u1 * np.cos(theta) + w1 * np.sin(theta)
        v2 =  v1
        w2 = -u1 * np.sin(theta) + w1 * np.cos(theta)
        return u2, v2, w2

    @staticmethod
    def _planar_fit_rotate(ur, vr, wr):
        """
        Planar-fit rotation: fit plane w = a u + b v and remove tilt,
        then rotate u-v to align mean wind.
        """
        # Fit w = a u + b v
        X = np.vstack([ur, vr]).T
        beta, _, _, _ = np.linalg.lstsq(X, wr, rcond=None)
        a, b = beta
        # remove plane tilt
        w1 = wr - (a * ur + b * vr)
        u1 = ur
        v1 = vr
        # then zero mean v1 via horizontal rotation
        phi = np.arctan2(np.mean(v1), np.mean(u1))
        u2 =  u1 * np.cos(phi) + v1 * np.sin(phi)
        v2 = -u1 * np.sin(phi) + v1 * np.cos(phi)
        return u2, v2, w1

    @staticmethod
    def preprocess(df, time_col, u_col, v_col, w_col, q_col, c_col,
                   avg_period=30, rot_method='double'):
        """
        Compute block-wise rotation and deviations (primes).

        Parameters
        ----------
        df : pandas.DataFrame
            High-frequency data with timestamp and raw u, v, w, q, c.
        time_col : str
            Timestamp column name (will be converted to DatetimeIndex).
        u_col, v_col, w_col, q_col, c_col : str
            Column names for raw variables.
        avg_period : int
            Averaging period in minutes (e.g. 30 for half-hour).
        rot_method : str
            'double' for double-rotation, 'planar' for planar-fit.

        Returns
        -------
        df_out : pandas.DataFrame
            Input DF plus:
              block_id       : block timestamp
              u_rot, v_rot, w_rot : rotated velocities
              u_mean, v_mean, w_mean, q_mean, c_mean : block means
              u_prime, v_prime, w_prime, q_prime, c_prime : deviations
        """
        df = df.copy()
        df[time_col] = pd.to_datetime(df[time_col])
        df = df.set_index(time_col)
        # assign block by floor
        freq = f'{avg_period}T'
        df['block_id'] = df.index.floor(freq)

        # rotate per block
        rotated = []
        for bid, grp in df.groupby('block_id'):
            ur = grp[u_col].values
            vr = grp[v_col].values
            wr = grp[w_col].values
            if rot_method == 'planar':
                u2, v2, w2 = CECPartitioner._planar_fit_rotate(ur, vr, wr)
            else:
                u2, v2, w2 = CECPartitioner._double_rotate(ur, vr, wr)
            rotated.append(pd.DataFrame({
                'u_rot': u2,
                'v_rot': v2,
                'w_rot': w2
            }, index=grp.index))
        rot_df = pd.concat(rotated)
        df = df.join(rot_df)

        # compute block means
        means = df.groupby('block_id')[['u_rot','v_rot','w_rot', q_col, c_col]].transform('mean')
        means.columns = ['u_mean','v_mean','w_mean','q_mean','c_mean']
        df = df.join(means)

        # deviations (primes)
        df['u_prime'] = df['u_rot'] - df['u_mean']
        df['v_prime'] = df['v_rot'] - df['v_mean']
        df['w_prime'] = df['w_rot'] - df['w_mean']
        df['q_prime'] = df[q_col]   - df['q_mean']
        df['c_prime'] = df[c_col]   - df['c_mean']

        df = df.reset_index()
        return df

    @staticmethod
    def partition(block_df, w_col='w_prime', q_col='q_prime', c_col='c_prime'):
        """
        Perform CEC partitioning on one block of deviations.

        Returns dict with ET, E, T, Fc, R, P, rET, rFc, ok.
        """
        w = block_df[w_col].values
        q = block_df[q_col].values
        c = block_df[c_col].values
        N = len(block_df)

        ET = np.mean(w * q)
        Fc = np.mean(w * c)

        IE = (w > 0) & (q > 0) & (c > 0)
        IT = (w > 0) & (q > 0) & (c < 0)

        fE = np.sum(IE * w * q) / N
        fT = np.sum(IT * w * q) / N
        fR = np.sum(IE * w * c) / N
        fP = np.sum(IT * w * c) / N

        rET = fE / fT if fT != 0 else np.nan
        rFc = fR / fP if fP != 0 else np.nan

        frac_oct = (IE.sum() + IT.sum()) / N
        ok = True
        if frac_oct < 0.20:
            ok = False
        if IE.sum() / N < 0.05:
            E, T = 0.0, ET
            R, P = 0.0, Fc
        elif IT.sum() / N < 0.05:
            E, T = ET, 0.0
            R, P = Fc, 0.0
        else:
            E = ET / (1 + 1/rET)
            T = ET / (1 + rET)
            R = Fc / (1 + 1/rFc)
            P = Fc / (1 + rFc)
        if np.isclose(rFc, -1.0, atol=0.1):
            ok = False

        return {
            'ET': ET, 'E': E, 'T': T,
            'Fc': Fc, 'R': R, 'P': P,
            'rET': rET, 'rFc': rFc,
            'ok': ok
        }

In [ ]:
# Read a TOA5 input file
def read_toa5_file(input_fn):
    df = pd.read_csv(input_fn,skiprows=[0,2,3], na_values=["NAN"])
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4], na_values=["NAN"])
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    #df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', errors="raise")
    df['timestamp'] = pd.to_datetime( df.timestamp, format='ISO8601', errors="raise")
    df.drop(columns=['RECORD'], inplace=True)
    return(df)

In [29]:
# Load data
df = read_toa5_file(data_fn)

# 1) Preprocess: rotate & compute 6 min means + deviations
df_dev = CECPartitioner.preprocess(
    df,
    time_col='timestamp',   # your timestamp column
    u_col='Ux',              # raw u
    v_col='Uy',              # raw v
    w_col='Uz',              # raw w
    q_col='H2O',              # raw water vapor
    c_col='CO2',              # raw CO₂
    avg_period=30,           # 6 min averaging
    rot_method='double'     # or 'planar'
)

# 2) Apply partitioning on each 6 min block
#    This returns one dict per block_id
results = (
    df_dev
    .groupby('block_id')
    .apply(lambda block: pd.Series(CECPartitioner.partition(block)))
    .reset_index()
)

In [33]:
display(df_dev)

display(results)

,timestamp,Ux,Uy,Uz,Ts,diag_sonic,CO2,H2O,diag_irga,Tc,...,u_mean,v_mean,w_mean,q_mean,c_mean,u_prime,v_prime,w_prime,q_prime,c_prime
0,2023-10-03 00:00:00.100,0.038125,0.731203,0.038794,14.35257,0,755.0154,8.405916,0,13.30579,...,0.326218,3.474611e-03,3.894473e-05,8.385039,758.519157,-0.573561,-0.693253,0.025332,0.020877,-3.503757
1,2023-10-03 00:00:00.200,0.024591,0.673369,0.035427,14.37440,0,754.1824,8.413427,0,13.32654,...,0.326218,3.474611e-03,3.894473e-05,8.385039,758.519157,-0.543917,-0.641703,0.023575,0.028388,-4.336757
2,2023-10-03 00:00:00.300,0.056279,0.632368,-0.023327,14.35780,0,754.4108,8.408866,0,13.31064,...,0.326218,3.474611e-03,3.894473e-05,8.385039,758.519157,-0.559334,-0.593353,-0.036107,0.023827,-4.108357
3,2023-10-03 00:00:00.400,0.077035,0.680085,0.028871,14.36020,0,754.3996,8.408824,0,13.31302,...,0.326218,3.474611e-03,3.894473e-05,8.385039,758.519157,-0.595657,-0.633144,0.014190,0.023785,-4.119557
4,2023-10-03 00:00:00.500,0.053584,0.621238,0.002381,14.33016,0,754.9482,8.407021,0,13.28342,...,0.326218,3.474611e-03,3.894473e-05,8.385039,758.519157,-0.554977,-0.583459,-0.010123,0.021982,-3.570957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865750,2023-10-03 23:59:59.600,-1.205956,-0.357506,-0.223951,18.42946,0,716.2968,7.725226,0,17.44113,...,1.426935,-3.083413e-16,-7.261938e-18,7.966859,725.861713,-0.165605,-0.153238,-0.133636,-0.241633,-9.564913
865751,2023-10-03 23:59:59.700,-1.208657,-0.372499,-0.167425,18.39159,0,717.1149,7.731947,0,17.40262,...,1.426935,-3.083413e-16,-7.261938e-18,7.966859,725.861713,-0.161232,-0.140567,-0.076648,-0.234912,-8.746813
865752,2023-10-03 23:59:59.800,-1.232592,-0.369121,-0.247880,18.33432,0,717.5883,7.739731,0,17.34476,...,1.426935,-3.083413e-16,-7.261938e-18,7.966859,725.861713,-0.134902,-0.153213,-0.155415,-0.227128,-8.273413
865753,2023-10-03 23:59:59.900,-1.219736,-0.479677,-0.156939,18.36469,0,716.9751,7.740038,0,17.37488,...,1.426935,-3.083413e-16,-7.261938e-18,7.966859,725.861713,-0.109209,-0.046706,-0.062386,-0.226821,-8.886613


,block_id,ET,E,T,Fc,R,P,rET,rFc,ok
0,2023-10-03 00:00:00,-0.001994,0.000000,-0.001994,0.391090,0.000000,0.391090,0.234637,-0.073546,True
1,2023-10-03 00:30:00,-0.000297,-0.000054,-0.000243,0.007003,-0.001832,0.008835,0.222415,-0.207374,True
2,2023-10-03 01:00:00,-0.000181,-0.000098,-0.000084,0.209778,0.486884,-0.277106,1.170281,-1.757031,True
3,2023-10-03 01:30:00,-0.006774,0.000000,-0.006774,0.641113,0.000000,0.641113,0.177878,-0.394444,False
4,2023-10-03 02:00:00,-0.000977,-0.000190,-0.000787,0.213555,-0.041223,0.254778,0.240991,-0.161800,True
5,2023-10-03 02:30:00,0.000454,0.000000,0.000454,-0.084646,0.000000,-0.084646,0.040399,-0.044811,True
6,2023-10-03 03:00:00,-0.001568,0.000000,-0.001568,0.102219,0.000000,0.102219,0.072251,-0.178819,False
7,2023-10-03 03:30:00,0.004659,0.000000,0.004659,-0.386904,0.000000,-0.386904,0.022150,-0.029928,True
8,2023-10-03 04:00:00,0.000459,0.000000,0.000459,-0.033137,0.000000,-0.033137,0.329583,-0.209070,True
9,2023-10-03 04:30:00,-0.002738,0.000000,-0.002738,0.298933,0.000000,0.298933,0.133858,-0.149060,False
